In [ ]:
import cv2
import numpy as np

# Initialize video capture (0 for webcam or replace with video file path)
cap = cv2.VideoCapture(0)

# Read the first frame
ret, frame1 = cap.read()
# Convert it to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# Apply Gaussian blur to reduce noise and improve motion detection
gray1 = cv2.GaussianBlur(gray1, (21, 21), 0)

while True:
    # Read the next frame
    ret, frame2 = cap.read()
    if not ret:
        break

    # Convert the current frame to grayscale
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to the new frame
    gray2 = cv2.GaussianBlur(gray2, (21, 21), 0)

    # Compute the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(gray1, gray2)

    # Apply a threshold to the difference to ignore small changes (e.g., noise)
    _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

    # Dilate the thresholded image to fill in holes and make contours more visible
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours of the detected motion
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around detected motion
    for contour in contours:
        if cv2.contourArea(contour) < 500:
            # Ignore small movements
            continue
        # Get the bounding box coordinates for each detected motion
        (x, y, w, h) = cv2.boundingRect(contour)
        # Draw the bounding box on the original frame
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the original frame with motion analysis
    cv2.imshow("Motion Detection", frame2)

    # Display the thresholded image (optional, for debugging)
    # cv2.imshow("Thresh", thresh)

    # Update the previous frame to be the current frame
    gray1 = gray2.copy()

    # Break the loop if 'q' is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()